# Task 5: Linear Models

_All credit for this jupyter notebook tutorial goes to the book "Hands-On Machine Learning with Scikit-Learn & TensorFlow" by Aurelien Geron. Modifications were made in preparation for the hands-on sessions._

# Setup

First, import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures:

In [ ]:
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Function to save a figure. This also decides that all output files 
# should stored in the subdirectorz 'classification'.
PROJECT_ROOT_DIR = "."
EXERCISE = "exercise_2b"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "output", EXERCISE, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Linear regression using the Normal Equation

Let's start with something simple: linear regression. As we learnt in the lecture, we can use the _normal equation_ to calculate the estimater theta_hat, an estimator for the parameter vector of the model. Let's start by generating some random data.

In [ ]:
import numpy as np

X = 2 * np.random.rand(100, 1)
y = 3 + 4 * X + np.random.randn(100, 1)

plt.plot(X, y, "b.")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([0, 2, 0, 15])
save_fig("generated_data_plot")
plt.show()

Looks good. Without using any machine learning yet, we can just use numpy's `linalg.inv()` function which inverts matrices. As we saw in the lecture, the estimator for theta can be calculated as the product of (the transposed vector of X times X) times the transposed vector X times the vector of target values. We will also use the `dot()` function to multiply matrices. And one more step is necessary: we need to append an additional feature $x_0 = 1$ to the feature vector before we start:

In [ ]:
X_b = np.c_[np.ones((100, 1)), X]  # add x0 = 1 to each instance
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
print(theta_best)

Ok, that's not perfect, but close enough. Where does the difference come from?

We can use this to make predictions of y values. Let's look at what the closed form would predict for data points at 0, 1 and 2:

In [ ]:
X_new = np.array([[0], [1], [2]])
X_new_b = np.c_[np.ones((3, 1)), X_new]  # add x0 = 1 to each instance
y_predict = X_new_b.dot(theta_best)
print(y_predict)

Are these accurate? Maybe it is more useful to plot the prediction as a line into the plot:

In [ ]:
plt.plot(X_new, y_predict, "r-", linewidth=2, label="Predictions")
plt.plot(X, y, "b.")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.legend(loc="upper right", fontsize=14)
plt.axis([0, 2, 0, 15])
save_fig("linear_model_predictions")
plt.show()

Linear regression can also be performed with Scikit-Learn and the `LinearRegression` class. We essentially only need to call its `fit()` function. This class uses a slightly more elaborate factorisation technique called _singular value decomposition_ to perform the invertion and matrix multiplication. This will scale better with high dimensionality of the matrix than calculating the inverse directly.

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, y)

print("Predicted theta_0: %s" % lin_reg.intercept_[0])
print("Predicted theta_1: %s" % lin_reg.coef_[0][0])

Let's try and see what our linear regression model predicts for the values at 0, 1 and 2:

In [ ]:
lin_reg.predict(X_new)

... which is almost the same as before.

# Linear regression using batch gradient descent

Let's try and implement linear regression with the batch gradient descent method! Quick reminder: gradient descent is an iterative approach to find the best estimate for our vector of model parameters theta. Using the learning rate eta, we iteratively adjust our estimates for theta in each learning step. The "direction" of adjustment is determined by the _gradient_ of the _mean square error_ of our theta values. Please have a look again at the slides of the lecture to find the definition.

Let's start with the following parameters and try to implement the iteration steps:

In [ ]:
eta = 0.1              # learning rate of 0.1
n_iterations = 1000    # 1000 iterations by default
m = 100                # number of data points in the training set
theta = np.random.randn(2,1)   # let's start with random values

for iteration in range(n_iterations):
    # Fill the missing lines here. We need to calculate
    # the vector of partial derivatives as well as the
    # new theta values.
    print("Values at iteration %s: %s, %s" % (iteration, theta[0][0], theta[1][0]))

Let's try and make a prediction with the obtained values for theta:

In [ ]:
X_new_b.dot(theta)

If we implement the same missing code piece in the function below, we can make plots to compare a few different learning rates. Try it out!

In [ ]:
def plot_gradient_descent(theta, eta):
    m = len(X_b)            # Determine size of dataset
    plt.plot(X, y, "b.")    # Plot the data points
    n_iterations = 1000     # Fix the number of iterations
    
    # Now loop over all iterations
    for iteration in range(n_iterations):
        # For the first few, plot the prediction line to get a trend.
        if iteration < 15:
            y_predict = X_new_b.dot(theta)
            style = "b-" if iteration > 0 else "r--"
            plt.plot(X_new, y_predict, style)
            
        # Here we need to put the above piece of code again.
        # [...]
        # gradients =
        # theta =
    plt.xlabel("$x_1$", fontsize=18)
    plt.axis([0, 2, 0, 15])
    plt.title(r"$\eta = {}$".format(eta), fontsize=16)


# Now let's plot some examples. First, start with random seeds for theta.
np.random.seed(42)
theta = np.random.randn(2,1)

plt.figure(figsize=(10,4))
plt.subplot(131); plot_gradient_descent(theta, eta=0.02)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.subplot(132); plot_gradient_descent(theta, eta=0.1)
plt.subplot(133); plot_gradient_descent(theta, eta=0.5)

save_fig("gradient_descent_plot")
plt.show()    

# Stochastic Gradient Descent

In [ ]:
theta_path_sgd = []
m = len(X_b)
np.random.seed(42)

In [ ]:
n_epochs = 50
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(m):
        if epoch == 0 and i < 20:                    # not shown in the book
            y_predict = X_new_b.dot(theta)           # not shown
            style = "b-" if i > 0 else "r--"         # not shown
            plt.plot(X_new, y_predict, style)        # not shown
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients
        theta_path_sgd.append(theta)                 # not shown

plt.plot(X, y, "b.")                                 # not shown
plt.xlabel("$x_1$", fontsize=18)                     # not shown
plt.ylabel("$y$", rotation=0, fontsize=18)           # not shown
plt.axis([0, 2, 0, 15])                              # not shown
save_fig("sgd_plot")                                 # not shown
plt.show()                                           # not shown

In [ ]:
theta

In [ ]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=50, tol=-np.infty, penalty=None, eta0=0.1, random_state=42)
sgd_reg.fit(X, y.ravel())

In [ ]:
sgd_reg.intercept_, sgd_reg.coef_

# Mini-batch gradient descent

In [ ]:
theta_path_mgd = []

n_iterations = 50
minibatch_size = 20

np.random.seed(42)
theta = np.random.randn(2,1)  # random initialization

t0, t1 = 200, 1000
def learning_schedule(t):
    return t0 / (t + t1)

t = 0
for epoch in range(n_iterations):
    shuffled_indices = np.random.permutation(m)
    X_b_shuffled = X_b[shuffled_indices]
    y_shuffled = y[shuffled_indices]
    for i in range(0, m, minibatch_size):
        t += 1
        xi = X_b_shuffled[i:i+minibatch_size]
        yi = y_shuffled[i:i+minibatch_size]
        gradients = 2/minibatch_size * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(t)
        theta = theta - eta * gradients
        theta_path_mgd.append(theta)

In [ ]:
theta

In [ ]:
theta_path_bgd = np.array(theta_path_bgd)
theta_path_sgd = np.array(theta_path_sgd)
theta_path_mgd = np.array(theta_path_mgd)

In [ ]:
plt.figure(figsize=(7,4))
plt.plot(theta_path_sgd[:, 0], theta_path_sgd[:, 1], "r-s", linewidth=1, label="Stochastic")
plt.plot(theta_path_mgd[:, 0], theta_path_mgd[:, 1], "g-+", linewidth=2, label="Mini-batch")
plt.plot(theta_path_bgd[:, 0], theta_path_bgd[:, 1], "b-o", linewidth=3, label="Batch")
plt.legend(loc="upper left", fontsize=16)
plt.xlabel(r"$\theta_0$", fontsize=20)
plt.ylabel(r"$\theta_1$   ", fontsize=20, rotation=0)
plt.axis([2.5, 4.5, 2.3, 3.9])
save_fig("gradient_descent_paths_plot")
plt.show()